# Option B - Generate Spheres from a CSV

In [1]:
out_dir = '/Users/cu135/Partners HealthCare Dropbox/Calvin Howard/studies/dti_case_report'
mask_path = '/Users/cu135/hires_backdrops/MNI/MNI152_T1_2mm_brain_mask.nii'

In [ ]:
# Specify the path to your CSV file containing NIFTI paths
input_csv_path = '/Users/cu135/Partners HealthCare Dropbox/Calvin Howard/resources/papez_nodes.csv'
sheet = None

In [ ]:
from calvin_utils.permutation_analysis_utils.statsmodels_palm import CalvinStatsmodelsPalm
# Instantiate the PalmPrepararation class
cal_palm = CalvinStatsmodelsPalm(input_csv_path=input_csv_path, output_dir=out_dir, sheet=sheet)
# Call the process_nifti_paths method
data_df = cal_palm.read_and_display_data()
data_df


# Preprocess Your Data

**Handle NANs**
- Set drop_nans=True is you would like to remove NaNs from data
- Provide a column name or a list of column names to remove NaNs from

In [ ]:
data_df.columns

In [ ]:
coordinate_col = ['list_of_coordinates']

In [ ]:
data_df = cal_palm.drop_nans_from_columns(columns_to_drop_from=coordinate_col)

Choose a radius (mm)

In [ ]:
radius = 3

A) Generate Spheres from a column that contains coordinates in lists. Ex) coord col: [x,y,z], [x,y,z]

In [ ]:
from calvin_utils.nifti_utils.generate_spherical_roi import SphereROIBuilder
roi_builder = SphereROIBuilder(mask_path=mask_path, out_dir=out_dir, radius=radius)
df = roi_builder.generate_from_coordlist_column(df=data_df, coord_col=coordinate_col[0], sub_col='sub', session_col=None, project_on_brain=True, save_individual_roi=False)
df

B) Generate Spheres from several columns that have X, Y, and Z in them. Ex) x_col: x | y_col: y | z_col: z

In [ ]:
from calvin_utils.nifti_utils.generate_spherical_roi import SphereROIBuilder
roi_builder = SphereROIBuilder(mask_path=mask_path, out_dir=out_dir, radius=radius)
df = roi_builder.generate_from_separate_coord_cols(data_df, x_col='', y_col='', z_col='', sub_col='', session_col=None, project_on_brain=True)
df

Save new DF with file paths

In [ ]:
out_dir

In [ ]:
df.to_csv(input_csv_path, index=False)

# Option A - Generate Sphere Manually

In [ ]:
out_dir = '/Users/cu135/Partners HealthCare Dropbox/Calvin Howard/studies/cognition_2023/revisions/notebook06/ongoing_ungodly_amount_of_edits'
mask_path = '/Users/cu135/hires_backdrops/MNI/MNI152_T1_2mm_brain_mask.nii'

Enter coordinates of sphere

In [5]:
x = -7
y = -46
z = 10
radius=8

Generate the Sphere at The Coordinate

In [6]:
import nibabel as nib
import numpy as np
brain_img = nib.load(mask_path)
mni_arr = np.array([x,y,z])

In [7]:
from calvin_utils.nifti_utils.generate_spherical_roi import SphereROIBuilder

SROI = SphereROIBuilder(mask_path=mask_path, out_dir=out_dir, radius=radius)
img = SROI.generate_from_single_coord(centre_xyz=[x,y,z],subject='sphere_boi', session=None, project_on_brain=True, suffix='')
img

'/Users/cu135/Partners HealthCare Dropbox/Calvin Howard/studies/dti_case_report/sub-sphere_boi/ses-01/roi/sub-sphere_boi_MNI152_T1_8.0mm_sphere_roi.nii.gz'

Done
- Calvin